**this is not right notebook the right notebook is on kaggle**

In [ ]:
!pip install pyarrow joblib tqdm


In [ ]:
import pandas as pd

df = pd.read_parquet(
    "/content/canonical_checkpoint_features_with_soh_and_split.parquet"
)

print(df.shape)
df.head()


(118770, 27)


,asset_id,cycle_id,V_mean,V_std,V_min,V_max,V_range,dV_dt_mean,dV_dt_max,T_mean,...,T_delta_base,duration_base,use_dV_dt,V_range_norm,T_delta_norm,duration_norm,dV_dt_norm,Degradation_Index,SOH_proxy,split
0,0,0,3.735076,0.245422,2.699819,4.191235,1.491416,-35.948592,38.111247,40.419340,...,1.335614,0.038896,True,0.692878,0.669490,0.729033,0.741807,0.708302,0.492480,train
1,0,1,3.735681,0.244972,2.699859,4.192679,1.492819,-36.342517,34.652440,40.327115,...,1.335614,0.038896,True,0.693348,0.688454,0.722873,0.693107,0.699446,0.496861,train
2,0,10,3.739525,0.240002,2.699924,4.192582,1.492658,-38.022390,41.584546,40.363359,...,1.335614,0.038896,True,0.693295,0.693133,0.699661,0.788435,0.718631,0.487419,train
3,0,11,3.739852,0.239651,2.699803,4.192502,1.492699,-38.328526,34.646588,40.433203,...,1.335614,0.038896,True,0.693308,0.711655,0.697407,0.693023,0.698848,0.497158,train
4,0,12,3.740231,0.238893,2.699924,4.192462,1.492538,-38.309224,34.655228,40.393250,...,1.335614,0.038896,True,0.693254,0.702437,0.695238,0.693147,0.696019,0.498566,train


In [ ]:
FEATURES = [
    "V_mean","V_std","V_min","V_max","V_range",
    "dV_dt_mean","dV_dt_max",
    "T_mean","T_max","T_delta",
    "duration_s"
]

TARGET = "SOH_proxy"


In [ ]:
df = df.sort_values(
    ["asset_id", "cycle_id"]
).reset_index(drop=True)


In [ ]:
WINDOW = 30


In [ ]:
import numpy as np
from tqdm import tqdm

def build_sequences(df, window):
    X, y, splits = [], [], []

    for asset_id, g in tqdm(df.groupby("asset_id")):
        g = g.reset_index(drop=True)

        for i in range(len(g) - window + 1):
            x_seq = g.loc[i:i+window-1, FEATURES].values
            y_val = g.loc[i+window-1, TARGET]
            split = g.loc[i+window-1, "split"]

            X.append(x_seq)
            y.append(y_val)
            splits.append(split)

    return np.array(X), np.array(y), np.array(splits)


In [ ]:
df_train = df[df["split"] == "train"].copy()


In [ ]:
def sequence_generator(df, window):
    for asset_id, g in df.groupby("asset_id"):
        g = g.sort_values("cycle_id").reset_index(drop=True)

        if len(g) < window:
            continue

        X = g[FEATURES].values
        y = g[TARGET].values

        for i in range(len(g) - window + 1):
            yield X[i:i+window], y[i+window-1]
